In [ ]:
#!pip install --user pandas

In [ ]:
import pandas as pd

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.datasets import load_boston

In [ ]:
b = load_boston()

In [ ]:
print(b.DESCR)

In [ ]:
df = pd.DataFrame(b.data, columns=b.feature_names)

In [ ]:
df['MEDV'] = b.target

In [ ]:
df[:3]

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
sns.boxplot(y=df['MEDV'])

In [ ]:
plt.scatter(df['CRIM'],df['MEDV'])

In [ ]:
plt.scatter(df['RM'],df['MEDV'])
plt.xlabel('Rooms')
plt.ylabel('Price')
plt.show()

In [ ]:
sns.lmplot(x='RM',y='MEDV', data=df)

In [ ]:
help(sns.lmplot)

In [ ]:
df[:5]

In [ ]:
# Assumption 1 --> Ensure that the x variables has a co-relation with Y variable

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(10,5))
sns.heatmap(df.corr(), annot=True)

In [ ]:
plt.scatter(df['LSTAT'],df['MEDV'])

In [ ]:
# Look for the distribution of the target variable

In [ ]:
sns.distplot(df['MEDV'])

In [ ]:
sns.boxplot(y=df['MEDV'])

In [ ]:
x = df.iloc[:,:13]

In [ ]:
y = df['MEDV']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.20, random_state=2)

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
x_train[:5]

In [ ]:
y_train[:5]

In [ ]:
sns.boxplot(y=y_train)

In [ ]:
sns.boxplot(y=df['MEDV'])

In [ ]:
# Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()

In [ ]:
model_lr = lr.fit(x_train,y_train)

In [ ]:
model_lr

In [ ]:
# statsmodel

In [ ]:
model_lr.coef_

In [ ]:
model_lr.intercept_

In [ ]:
model_lr.predict(x_test.iloc[:1,:])

In [ ]:
x_test.iloc[:1,:]

In [ ]:
y_test.iloc[:5]

In [ ]:
#e(y)=37.19980774264999+(-1.08223557e-01)*(5.82115)+ 3.89039349e-02*(0)+(-1.51517627e-02)*(18.1)+.....xn

In [ ]:
pred_price = model_lr.predict(x_test)

In [ ]:
pred_price[:5]

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(y_test,pred_price)

In [ ]:
import numpy as np

In [ ]:
np.sqrt(mean_squared_error(y_test,pred_price))

In [ ]:
sns.lineplot(y_test,pred_price)

In [ ]:
error_df = pd.DataFrame({'Actual':y_test,'Predicted':pred_price})

In [ ]:
err=error_df.reset_index(drop=True)

In [ ]:
err.plot()

In [ ]:
df.to_csv('boston.csv')

In [ ]:
import findspark
findspark.init('/usr/local/spark')

In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('Regression').getOrCreate()

In [ ]:
df1 = spark.createDataFrame(df)

In [ ]:
df1.head()

In [ ]:
df1.show(2)

In [ ]:
df2 = spark.read.csv('boston.csv', inferSchema=True,header=True)

In [ ]:
df4=df1.toPandas()

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
for i in df1.columns:
    print(i, end=',')

In [ ]:
assmblr = VectorAssembler(inputCols=['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B'
                                     ,'LSTAT'],outputCol="features")

In [ ]:
output = assmblr.transform(df1)

In [159]:
output.show(3)

+-------+----+-----+----+-----+-----+----+------+---+-----+-------+------+-----+----+--------------------+
|   CRIM|  ZN|INDUS|CHAS|  NOX|   RM| AGE|   DIS|RAD|  TAX|PTRATIO|     B|LSTAT|MEDV|            features|
+-------+----+-----+----+-----+-----+----+------+---+-----+-------+------+-----+----+--------------------+
|0.00632|18.0| 2.31| 0.0|0.538|6.575|65.2|  4.09|1.0|296.0|   15.3| 396.9| 4.98|24.0|[0.00632,18.0,2.3...|
|0.02731| 0.0| 7.07| 0.0|0.469|6.421|78.9|4.9671|2.0|242.0|   17.8| 396.9| 9.14|21.6|[0.02731,0.0,7.07...|
|0.02729| 0.0| 7.07| 0.0|0.469|7.185|61.1|4.9671|2.0|242.0|   17.8|392.83| 4.03|34.7|[0.02729,0.0,7.07...|
+-------+----+-----+----+-----+-----+----+------+---+-----+-------+------+-----+----+--------------------+
only showing top 3 rows



In [ ]:
output.select(['features']).show()

In [ ]:
final_df = output.select("features","MEDV")

In [ ]:
train_data,test_data = final_df.randomSplit([0.8,0.2])

In [158]:
train_data.count()

410

In [ ]:
test_data.count()

In [ ]:
train_data.show(4)

In [ ]:
help(final_df.randomSplit)

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
lr = LinearRegression(labelCol='MEDV')

In [ ]:
model=lr.fit(train_data)

In [ ]:
model.coefficients

In [ ]:
pred_price = model.evaluate(test_data)

In [ ]:
pred_price.predictions.show(5)

In [ ]:
pred_price.rootMeanSquaredError